In [23]:
import pandas as pd
import numpy as np
import datetime

In [24]:
pc = pd.read_csv("household_power_consumption.txt", sep = ";", low_memory = False)
print(pc.info())
##
# Check null values
##
print(len(pc) - pc.count())
## Only a very small percentage of Sub_metering_3 values are null.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
Date                     object
Time                     object
Global_active_power      object
Global_reactive_power    object
Voltage                  object
Global_intensity         object
Sub_metering_1           object
Sub_metering_2           object
Sub_metering_3           float64
dtypes: float64(1), object(8)
memory usage: 142.5+ MB
None
Date                         0
Time                         0
Global_active_power          0
Global_reactive_power        0
Voltage                      0
Global_intensity             0
Sub_metering_1               0
Sub_metering_2               0
Sub_metering_3           25979
dtype: int64


In [25]:
print(pc[pc['Sub_metering_3'].isna()])
# That's sure interesting. It looks like we have a lot more 
# missing values than expected. There are ? marks in other fields
# rather than nan's. Many of the times are adjacent as well.

               Date      Time Global_active_power Global_reactive_power  \
6839     21/12/2006  11:23:00                   ?                     ?   
6840     21/12/2006  11:24:00                   ?                     ?   
19724    30/12/2006  10:08:00                   ?                     ?   
19725    30/12/2006  10:09:00                   ?                     ?   
41832     14/1/2007  18:36:00                   ?                     ?   
61909     28/1/2007  17:13:00                   ?                     ?   
98254     22/2/2007  22:58:00                   ?                     ?   
98255     22/2/2007  22:59:00                   ?                     ?   
142588    25/3/2007  17:52:00                   ?                     ?   
190497    28/4/2007  00:21:00                   ?                     ?   
190498    28/4/2007  00:22:00                   ?                     ?   
190499    28/4/2007  00:23:00                   ?                     ?   
190500    28/4/2007  00:2

In [26]:
def check_non_float(a):
    try:
        float(a)
        return(False)
    except Exception as e:
        return(True)
    return(True)

nn_index = pc.loc[ :, (pc.columns != 'Date') & (pc.columns != 'Time')].applymap(check_non_float)

nn_set = []
for column in nn_index.columns:
    nn_set += list(pc[column][nn_index[column]])

nn_set = set(nn_set)
print(nn_set)

# Wow, that's wonderful. It looks like the only non-float character 
# is a question mark. For simplicity, I will replace all nans with a
# question mark.

pc = pc.fillna('?')

pc['q_count'] = pc.apply(lambda x: sum([1 if a == '?' else 0 for a in x]), axis = 1)


{'?'}


In [27]:
print(sum([1 if a != 0 else 0 for a in pc['q_count']]))

# List unique numbers non-numeric characters by row:
print(set(list(pc['q_count'])))

# Wow, that is super interesting. It seems that all rows that have
# any missing data are missing all data.

# Now I need to check how close in proximity to time these missing values are

timestamps = pc.apply(lambda x: datetime.datetime.strptime(x['Date'].split('/')[2] + '/' + x['Date'].split('/')[1] + '/' + x['Date'].split('/')[0] + ' ' + x['Time'], "%Y/%m/%d %H:%M:%S"), axis = 1)
pc['timestamp'] = timestamps



25979
{0, 7}


In [51]:
# Check to see if times are already sorted:

print(''.join([str(a) for a in pc.index.values]) == ''.join([str(a) for a in pc.sort_values(by = ['timestamp']).index.values]))

# Great! The dataframe is already sorted by time.

# Now, let's check if there is exactly one row per minute.

time_diff = max(pc['timestamp']) - min(pc['timestamp'])
print(len(set(pc['timestamp'])))

# Timestamps are all unique.

print(time_diff.days * 24 * 60 + time_diff.seconds / 60)

# There are the same number of minutes as rows.

print(set([(a-datetime.datetime(1970,1,1)).total_seconds() % 60 for a in pc['timestamp']]))

# Every row represents a whole minute.

# Haha! It looks like there is exactly one row per minute.
# Each minute is consecutive
# Time differences will be easy to calculate.



True
2075259
2075258.0
{0.0}


In [76]:
pc['consecutive_minutes'] = list(range(len(pc)))

missing_pc = pc[pc['q_count'] > 0]
time_diffs = list([str(int(a)) for a in missing_pc['cumulative_minutes'].diff()[1:]])
data_loss_intervals = [len(a) for a in ''.join([a if a == '1' else '0' for a in time_diffs]).split('0') if a != '']
print('There were ' + str(len(data_loss_intervals)) + ' intervals of data loss.')
print('The longest duration between missing intervals lasted ' + str(round(int(max(time_diffs)) / 60 / 24,2)) + ' days.')
print('The maximum length of continuous data loss lasted ' + str(round(max(data_loss_intervals) / 60, 2)) + ' hours.')
print('The minimum length of continuous data loss lasted ' + str(min(data_loss_intervals)) + ' minute.')
print('The median length of continuous data loss lasted ' + str(int(np.median(data_loss_intervals))) + ' minutes.')



There were 33 intervals of data loss.
The longest duration between missing intervals lasted 54.66 days.
The maximum length of continuous data loss lasted 120.42 hours.
The minimum length of continuous data loss lasted 1 minute.
The median length of continuous data loss lasted 3 minutes.


In [82]:
pc_rmna = pc[pc['Global_active_power'] != '?']
print(len(pc))
print(len(pc_rmna))
print(len(pc_rmna) - pc_rmna.count())

# There are no longer any missing values. Save to csv.
print(pc_rmna.columns.values)
pc_rmna[['Date', 'Time', 'Global_active_power', 'Global_reactive_power', 'Voltage',
 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].to_csv('pc_rmna.csv', index = False)

2075259
2049280
Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
q_count                  0
timestamp                0
cumulative_minutes       0
consecutive_minutes      0
dtype: int64
['Date' 'Time' 'Global_active_power' 'Global_reactive_power' 'Voltage'
 'Global_intensity' 'Sub_metering_1' 'Sub_metering_2' 'Sub_metering_3'
 'q_count' 'timestamp' 'cumulative_minutes' 'consecutive_minutes']
